In [422]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from glob import glob
import json
import folium
import warnings
import requests
from urllib.parse import urlparse, quote
import matplotlib as mpl
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore',category=FutureWarning)
%matplotlib inline
mpl.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus=False)

### opinet에서 동 데이터 가져오기

In [423]:
driver = webdriver.Chrome('D:/workspace/data-home/05.Crawling/chromedriver.exe')
driver.get('http://www.opinet.co.kr/user/main/mainView.do')
driver.maximize_window()
driver.find_element_by_css_selector('.ic_m1').click()

In [424]:
# 집 화곡동과 근처 등촌동,내발산동
driver.find_element_by_xpath('//*[@id="SIDO_NM0"]').send_keys('서울')   
driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]').send_keys('강서구')
region = driver.find_element_by_xpath('//*[@id="DONG_NM"]')
dong_list = region.find_elements_by_tag_name('option')
dong_names = [dong.get_attribute('value') for dong in dong_list][1:]
dong_name = [dong for dong in dong_names if '화곡' in dong or '등촌' in dong or '내발산' in dong]
dong_name

['내발산동',
 '등촌동',
 '등촌제1동',
 '등촌제2동',
 '등촌제3동',
 '화곡동',
 '화곡본동',
 '화곡제1동',
 '화곡제2동',
 '화곡제3동',
 '화곡제4동',
 '화곡제5동',
 '화곡제6동',
 '화곡제7동',
 '화곡제8동']

### 가져온 동 데이터로 주유소정보 저장

In [425]:
driver.find_element_by_xpath('//*[@id="SIDO_NM0"]').send_keys('서울')   
driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]').send_keys('강서구')
for dong in dong_name:
    print(dong)
    driver.find_element_by_xpath('//*[@id="DONG_NM"]').send_keys(dong)
    driver.find_element_by_xpath('//*[@id="searRgSelect"]/span').click()
    time.sleep(1)
    if driver.find_element_by_xpath('//*[@id="os_price1"]/table/tbody/tr/td').text != '검색된 결과가 없습니다.' :
        driver.find_element_by_xpath('//*[@id="glopopd_excel"]/span').click()
    time.sleep(1)
driver.close()


내발산동
등촌동
등촌제1동
등촌제2동
등촌제3동
화곡동
화곡본동
화곡제1동
화곡제2동
화곡제3동
화곡제4동
화곡제5동
화곡제6동
화곡제7동
화곡제8동


### 저장된 파일 가져오기

In [426]:
stations_files = glob('C:/Users/SU/Downloads/지역*.xls')
stations_files

['C:/Users/SU/Downloads\\지역_위치별(주유소) (1).xls',
 'C:/Users/SU/Downloads\\지역_위치별(주유소) (2).xls',
 'C:/Users/SU/Downloads\\지역_위치별(주유소) (3).xls',
 'C:/Users/SU/Downloads\\지역_위치별(주유소) (4).xls',
 'C:/Users/SU/Downloads\\지역_위치별(주유소).xls']

### 가져온 파일 데이터 프레임으로 만들기

In [427]:
tmp_raw = []
for file in stations_files:
    tmp = pd.read_excel(file,header=2)
    tmp_raw.append(tmp)
df = pd.concat(tmp_raw)
df = df.reset_index()
del df['index']
df

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,현대오일뱅크 등촌주유소,서울 강서구 등촌로 189 (등촌동),현대오일뱅크,02-2647-5700,N,-,1317,1127,-
1,서울특별시,코리나(주)청룡주유소,서울 강서구 양천로 576 (등촌동),SK에너지,02-3661-1575,N,-,1398,1208,1100
2,서울특별시,강서열린점,서울 강서구 등촌로 213 (등촌동),GS칼텍스,02-2647-9091,N,-,1327,1127,1250
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-
4,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-
5,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100
6,서울특별시,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,02-2654-4564,Y,-,1266,1072,-
7,서울특별시,신화곡주유소,서울 강서구 국회대로 71,S-OIL,02-2611-3161,Y,-,1315,1115,-
8,서울특별시,화곡현대주유소,서울 강서구 강서로 151 (화곡동),SK에너지,02-2607-1942,Y,1455,1326,1126,-
9,서울특별시,하이웨이주유소,서울특별시 강서구 공항대로 432(화곡동),S-OIL,02-2605-4000,Y,1457,1287,1127,-


### 돌릴때마다 다운로드에서 지우는게 귀찮아서 자동으로 지우게 만들었습니다

In [428]:
# xls파일들 지우기
for i in stations_files:
    if os.path.isfile(i):   # 파일이 있는지 먼저확인(파일없이 삭제시키면 오류)
        os.remove(i)

### 위도 경도 추가하기

In [429]:
key_fd = open('./05.Crawling/kmap_key.txt')
kmap_key = key_fd.read(100)
key_fd.close()
lat_list=[]
lng_list=[]
for i in df.index:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + df['주소'][i]
    print(url)
    result = requests.get(urlparse(url).geturl(), headers={"Authorization":"KakaoAK "+ kmap_key}).json()
    address = result['documents'][0]['address']
    lng = address['x'] 
    lat = address['y']
    lng_list.append(lng)
    lat_list.append(lat)


https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 등촌로 189 (등촌동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 양천로 576 (등촌동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 등촌로 213 (등촌동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 곰달래로 207(화곡동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울특별시 강서구  강서로 154 (화곡동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 국회대로 225 (화곡동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 국회대로 251 (화곡동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 국회대로 71
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 강서로 151 (화곡동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울특별시 강서구 공항대로 432(화곡동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 화곡로 273 (화곡동)
https://dapi.kakao.com/v2/local/search/address.json?query=서울 강서구 강서로 292 (내발산동)
https://dapi.kakao.com/v2/local/search/address.j

In [430]:
df['lng'] = lng_list
df['lat'] = lat_list

In [431]:
type(df['lng'][0])

str

#### 동내에서 제일 싼곳

In [432]:
for i in df.index:
    df.lat[i] = pd.to_numeric(df.lat[i])
    df.lng[i] = pd.to_numeric(df.lng[i])
low_price = df.sort_values(by='휘발유')[:1][['지역','상호','주소','휘발유']]
high_price = df.sort_values(by='휘발유',ascending=False)[:1][['지역','상호','주소','휘발유']]
low_price

,지역,상호,주소,휘발유
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243


#### 동네에서 가장 비싼곳

In [433]:
high_price

,지역,상호,주소,휘발유
1,서울특별시,코리나(주)청룡주유소,서울 강서구 양천로 576 (등촌동),1398


### 집 좌표 구하기

In [434]:
# 집근처가 강서구청이라 강서구청을 기준으로 잡았습니다
serch = '강서구청'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + quote(serch)
result = requests.get(urlparse(url).geturl(), headers={"Authorization":"KakaoAK "+ kmap_key}).json()
address = result['documents'][0]
home_x = address['x']
home_y = address['y']
home_x,home_y

('126.849533759513', '37.5509646154307')

### 지도위에 마커로 표시하기

In [435]:
mapping = folium.Map(location=[home_y, home_x],zoom_start=14.4)
folium.CircleMarker([home_y, home_x], 
                    radius=80,                 
                    color='cadetblue',            
                    popup='').add_to(mapping)
folium.CircleMarker([home_y, home_x], 
                    radius=230,                 
                    color='cadetblue',            
                    popup='').add_to(mapping)
for i in df.index:
    folium.Marker([float(df.lat[i]),float(df.lng[i])],
                   popup=f'''{df['상호'][i]}, {df['휘발유'][i]}''',
                   icon=folium.Icon(color = 'red' if df['휘발유'][i] == df['휘발유'].min() else 'blue',
                   icon='thumbs-up' if df['휘발유'][i] == df['휘발유'].min() else 'tint'),
                   tooltip=df['휘발유'][i]).add_to(mapping)
mapping

In [439]:
help(folium.Icon)

Help on class Icon in module folium.map:

class Icon(branca.element.MacroElement)
 |  Icon(color='blue', icon_color='white', icon='info-sign', angle=0, prefix='glyphicon', **kwargs)
 |  
 |  Creates an Icon object that will be rendered
 |  using Leaflet.awesome-markers.
 |  
 |  Parameters
 |  ----------
 |  color : str, default 'blue'
 |      The color of the marker. You can use:
 |  
 |          ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 |           'gray', 'black', 'lightgray']
 |  
 |  icon_color : str, default 'white'
 |      The color of the drawing on the marker. You can use colors above,
 |      or an html color code.
 |  icon : str, default 'info-sign'
 |      The name of the marker sign.
 |      See Font-Awesome website to choose yours.
 |      Warning : depending on the icon you choose you may need to adapt
 |      the `